In [ ]:
import os
import pandas as pd

In [ ]:
dca_data = pd.read_csv("processed/combined.csv")
dca_data

In [ ]:
mmsi_data = pd.read_excel("data/MMSI_rc_20211027_.xlsx")
mmsi_link = mmsi_data[["mmsi", "kallesignal"]]
mmsi_link



In [ ]:

dca_mmsi = dca_data.merge(mmsi_link, left_on="Radiokallesignal (ERS)", right_on="kallesignal").drop(columns=["kallesignal"])
dca_mmsi["Starttidspunkt"] = pd.to_datetime(dca_mmsi["Starttidspunkt"])
dca_mmsi["Stopptidspunkt"] = pd.to_datetime(dca_mmsi["Stopptidspunkt"])
dca_date_slice = dca_mmsi[["Starttidspunkt", "Stopptidspunkt", "mmsi"]]


In [ ]:
column_dtypes = {"mmsi": int, "date_time_utc": object, "lon": float, "lat": float, "sog": float, "cog": float, "true_heading": int, "nav_status": int, "message_nr": int}
ais_test = pd.read_csv("data/ais_data/ais2015/ais_20150101.csv", sep=";", dtype=column_dtypes)
# ais_test = pd.read_csv("data/ais_data/ais2015/ais_20150101.csv", sep=";")
ais_test["date_time_utc"] = pd.to_datetime(ais_test["date_time_utc"])
ais_test

In [ ]:
def in_interval(ais_date, **kwargs):
    # is_in = dca_date_slice.apply(lambda x: ais_date >= x["Starttidspunkt"] and ais_date <= x["Stopptidspunkt"], axis=1)
    stop = ((dca_date_slice["Stopptidspunkt"] >= ais_date))# and (dca_date_slice["Stopptidspunkt"] >= ais_date))
    # print(stop)
    start = ((dca_date_slice["Starttidspunkt"] <= ais_date))# and (dca_date_slice["Stopptidspunkt"] >= ais_date))
    # print(start)
    is_in = start & stop
    
    return is_in.any()
# test = in_interval(pd.to_datetime("2012-01-01 23:33:52"))

In [ ]:
# Function to process each chunk and determine if the dates fall within any intervals
def process_chunk(chunk, start_times, stop_times):
    ais_dates = chunk['date_time_utc'].values

    # Use broadcasting to create a boolean array where True means the date is in the interval
    is_in_start = ais_dates[:, None] >= start_times
    is_in_stop = ais_dates[:, None] <= stop_times
    is_in_interval = is_in_start & is_in_stop

    # Any interval containing the ais_date will have True in the row
    chunk['fishing'] = is_in_interval.any(axis=1)
    return chunk

# Convert interval times to numpy arrays
start_times = dca_date_slice['Starttidspunkt'].values
stop_times = dca_date_slice['Stopptidspunkt'].values

# Define the chunk size
chunk_size = 10000  # Adjust based on memory constraints

# Process ais_test in chunks
chunks = []
for start in range(0, len(ais_test), chunk_size):
    print(f"{start}/{len(ais_test)}")
    end = start + chunk_size
    chunk = ais_test.iloc[start:end].copy()
    processed_chunk = process_chunk(chunk, start_times, stop_times)
    chunks.append(processed_chunk)

# Concatenate all chunks back together
result = pd.concat(chunks, ignore_index=True)

print(result)


In [ ]:
result["fishing"].describe()

In [ ]:
ais_test["fishing"] = ais_test["date_time_utc"].apply(in_interval, axis=1)

In [ ]:
def read_ais(ais_folder):

    ais_days = []
    
    for day in os.listdir(ais_folder):
        if day.endswith(".csv"):
            ais = pd.read_csv(os.path.join(ais_folder, day), sep=";", dtype=column_dtypes)
            ais_days.append(ais)

    ais_df = pd.concat(ais_days)
    return ais_df
                          

# ais_df = read_ais("data/ais_data/ais2015/")
    